# Math Concepts for Developers - April 2019 Exam SoftUni 2019
# Radoslav Ivanov 
## Recommendations Using Matrix Factorization

### Problem

We have data that comes from previous purchasing of different users or some ratings of movies, for example, and we want to recommend a new product or a new film to a user, based on other products and users' ratings. In this case, all the information we have about the existing ratings can be represented in a matrix. Assume now that we have 5 users and 4 items, and ratings are integers ranging from 1 to 5, the matrix may look something like this:

| - | D1 | D2 | D3 | D4 |
| --- | --- | --- |--- | --- |
| U1 | 5 | 3 | - | 1 |
| U2 | 4 | - | - | 1 |
| U3 | 1 | 1 | - | 5 |
| U4 | 1 | - | - | 4 |
| U5 | - | 1 | 5 | 4 |

Matrix factorization is to factorize a matrix, i.e. to find out two (or more) matrices such that when you multiply them you will get back the original matrix. Matrix factorization techniques are usually more effective than user-based or item-based collaborative filtering methods, because they allow us to discover the latent features underlying the interactions between users and items. If we can discover these latent features, we should be able to predict a rating with respect to a certain user and a certain item, because the features associated with the user should match with the features associated with the item. In trying to discover the different features, we also make the assumption that the number of features would be smaller than the number of users and the number of items.



### The mathematics of matrix factorization

We have a set U of users, and a set D of items. Let 
$\mathbf{R}$ of size $|U| \times |D|$ be the matrix that contains all the ratings that the users have assigned to the items. Also, we assume that we would like to discover $K$ latent features. Our task, then, is to find two matrics matrices $\mathbf{P} (a |U| \times K matrix)$ and $\mathbf{Q} (a |D| \times K matrix)$ such that their product approximates $\mathbf{R}$:

$\mathbf{R} \approx \mathbf{P} \times \mathbf{Q}^T = \hat{\mathbf{R}}$

The error between the estimated rating and the real rating, can be calculated by the following equation for each user-item pair:



$e_{ij}^2 = (r_{ij} - \hat{r}_{ij})^2 = (r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2$

### Regularization

A common extension to this basic algorithm is to introduce regularization to avoid overfitting. This is done by adding a parameter 
$\beta$ and modify the squared error as follows:

$e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||P||^2 + ||Q||^2)}$

$\alpha$ is a constant whose value determines the rate of approaching the minimum. Usually we will choose a small value for , say 0.0002. This is because if we make too large a step towards the minimum we may run into the risk of missing the minimum and end up oscillating around the minimum.  In practice, $\beta$ is set to some values in the range of 0.02. The new update rules for this squared error can be:

$p'_{ik} = p_{ik} + \alpha \frac{\partial}{\partial p_{ik}}e_{ij}^2 = p_{ik} + \alpha(2 e_{ij} q_{kj} - \beta p_{ik} )$

$q'_{kj} = q_{kj} + \alpha \frac{\partial}{\partial q_{kj}}e_{ij}^2 = q_{kj} + \alpha(2 e_{ij} p_{ik} - \beta q_{kj} )$

### Implementation 

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from scipy import sparse

In [3]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T


Sample with small matrix

In [4]:
    R = [
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ]

    R = np.array(R)

    N = len(R)
    M = len(R[0])
    K = 3

    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K)
    nR = np.dot(nP, nQ.T)
    print(nR)

[[4.98411474 2.9625321  4.23220164 1.00003899]
 [3.97529924 2.08356978 3.49676554 0.99826035]
 [1.02407506 0.93980712 4.96631313 4.96530826]
 [0.98750795 0.4176592  4.01554481 3.97756223]
 [2.37239165 1.07622222 4.95575732 4.01165085]]


In [5]:
def displayResults(nR,R):
    result = nR - R
    for index in range(len(result)):
        print("For user " + str(index) + " max predicted score " + str(max(result[index])) +" for film number " + str(np.argmax(result[index])))

In [6]:
displayResults(nR,R)

For user 0 max predicted score 4.232201636391201 for film number 2
For user 1 max predicted score 3.4967655422297366 for film number 2
For user 2 max predicted score 4.966313130169251 for film number 2
For user 3 max predicted score 4.015544812602086 for film number 2
For user 4 max predicted score 2.3723916547420063 for film number 0


In [7]:
# read in triples of user/artist/playcount from the input dataset
data = pd.read_csv("ml/ratings.csv")
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## More data
Let's try with more data. For matrix 4x5 is fast, but what happens with matrix 40x50(slow), 400x500 (very slow) and 4000 x 5000 on CPU?



---------


In [8]:
df_movies = pd.read_csv("ml/movies.csv",usecols=['movieId', 'title'], dtype={'movieId': 'int32', 'title': 'str'})
print(df_movies.shape)
df_movies.head()

(9742, 2)


,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [9]:
df_ratings = pd.read_csv("ml/ratings.csv",usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})
print(df_movies.shape)
df_ratings.head()

(9742, 2)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
# create a sparse matrix of all the artist/user/play triples
plays = sparse.coo_matrix((data['rating'], 
                   (data['movieId'], 
                    data['userId'])))

In [11]:
df_movies_cnt = pd.DataFrame(df_ratings.groupby('movieId').size(), columns=['count'])
popular_movies = list(set(df_movies_cnt.query('count >= 0').index))  # noqa
movies_filter = df_ratings.movieId.isin(popular_movies).values
df_users_cnt = pd.DataFrame(df_ratings.groupby('userId').size(),columns=['count'])
active_users = list(set(df_users_cnt.query('count >= 0').index))  # noqa
users_filter = df_ratings.userId.isin(active_users).values
df_ratings_filtered = df_ratings[movies_filter & users_filter]
# pivot and create movie-user matrix
movie_user_mat = df_ratings_filtered.pivot(index='userId', columns='movieId', values='rating').fillna(0)
print(movie_user_mat.shape)
movie_user_mat.head()

(610, 9724)


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
shortmatrix = movie_user_mat.loc[:50, :43]
shortmatrix

movieId,1,2,3,4,5,6,7,8,9,10,...,31,32,34,36,38,39,40,41,42,43
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,3.0,4.0,4.0,5.0,0.0,0.0,0.0,4.0,0.0,4.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,3.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0


In [13]:
R = shortmatrix

R = np.array(R)

N = len(R)
M = len(R[0])
K = 30

P = np.random.rand(N,K)
Q = np.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)
nR = np.dot(nP, nQ.T)
print(nR)


[[3.99155133 4.29784082 3.99427955 ... 5.01082194 5.32149811 4.65516393]
 [4.71992028 5.32493527 5.51661215 ... 5.50458305 7.39561551 5.93679409]
 [3.42925254 4.06107984 3.38408445 ... 3.78600772 3.72504827 3.33322375]
 ...
 [5.11014729 5.96794791 5.13829381 ... 5.88914831 8.27530087 6.36203303]
 [4.79769906 5.23215075 4.91014217 ... 4.66665562 7.56010006 5.63974783]
 [2.99930273 3.61497576 4.62991542 ... 4.52517682 6.53725634 3.79195902]]


In [14]:
displayResults(nR,R)

For user 0 max predicted score 7.544209412593386 for film number 36
For user 1 max predicted score 9.392504855861883 for film number 29
For user 2 max predicted score 6.308842392929963 for film number 29
For user 3 max predicted score 5.819379682876541 for film number 27
For user 4 max predicted score 6.872819901509884 for film number 27
For user 5 max predicted score 6.365462773405296 for film number 36
For user 6 max predicted score 7.593348282872636 for film number 29
For user 7 max predicted score 7.5309291567395436 for film number 29
For user 8 max predicted score 7.871460183626262 for film number 27
For user 9 max predicted score 8.314907444090426 for film number 27
For user 10 max predicted score 6.883769073509757 for film number 27
For user 11 max predicted score 8.91788063642687 for film number 29
For user 12 max predicted score 9.120937898988176 for film number 8
For user 13 max predicted score 5.521861492759424 for film number 17
For user 14 max predicted score 6.12456773425

In [15]:
smallmatrix = movie_user_mat.loc[:500, :458]
smallmatrix


movieId,1,2,3,4,5,6,7,8,9,10,...,449,450,451,452,453,454,455,456,457,458
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,0.0,4.0,0.0,0.0,0.0,4.0,3.0,0.0,5.0,3.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
R = smallmatrix

R = np.array(R)

N = len(R)
M = len(R[0])
K = 300

P = np.random.rand(N,K)
Q = np.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)
nR = np.dot(nP, nQ.T)
print(nR)


KeyboardInterrupt: 

### Pros and Cons
We can make recommendations Using Matrix Factorization for small matrix and it is working as expected, but if we have more data and we need to add new a movie or a user we need to recalculate matrix again and again. On CPU we need a lot more time.
One optimization is to use GPU, the other is to use algorithm such as k-nearest neighbors (KNN) and neural network for big data.

### References:
1. Matrix Factorization: A Simple Tutorial and Implementation in Python http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/
2. Prototyping a Recommender System Step by Step Part 1: KNN Item-Based Collaborative Filtering https://github.com/KevinLiao159/MyDataSciencePortfolio/blob/master/movie_recommender/src/knn_recommender.py 